In [1]:
# import & connect google drive

from google.colab import drive
import cv2
import numpy as np
import os
import pandas as pd

# === Mount Drive ===
drive.mount('/content/drive')

Mounted at /content/drive
KakaoTalk_20250721_171314402_04.jpg: ✅ 56.13 cm²
KakaoTalk_20250721_171314402_01.jpg: ✅ 88.05 cm²
KakaoTalk_20250721_171314402_06.jpg: ✅ 52.97 cm²
KakaoTalk_20250721_171314402_09.jpg: ✅ 86.78 cm²
KakaoTalk_20250721_171314402_11.jpg: ❌ marker not detected

✅ Analysis complete


In [ ]:
# Marker check: white & round

MARKER_REAL_AREA_CM2 = 12.57
IMAGE_DIR = '/content/drive/MyDrive/ab'
OUTPUT_DIR = os.path.join(IMAGE_DIR, 'results')
os.makedirs(OUTPUT_DIR, exist_ok=True)

# === HSV ranges ===
WHITE_LOWER = np.array([90, 5, 160])
WHITE_UPPER = np.array([125, 100, 255])
PLANT_LOWER = np.array([20, 40, 40])
PLANT_UPPER = np.array([90, 255, 255])

# === Marker detection ===
def detect_marker(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, WHITE_LOWER, WHITE_UPPER)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5,5), np.uint8), iterations=2)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None

    c = max(contours, key=cv2.contourArea)
    area = cv2.contourArea(c)
    peri = cv2.arcLength(c, True)
    if peri == 0:
        return None

    circularity = 4 * np.pi * area / (peri ** 2)
    (x, y), r = cv2.minEnclosingCircle(c)
    if circularity < 0.75:
        return None

    return area, (int(x), int(y), int(r))

In [ ]:
# Plant check: green & biggest lump

def detect_plant(hsv):
    mask = cv2.inRange(hsv, PLANT_LOWER, PLANT_UPPER)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((7,7), np.uint8), iterations=2)

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None

    c = max(contours, key=cv2.contourArea)
    return c, cv2.contourArea(c)

In [ ]:
# Image analysis

def analyze_image(path):
    image = cv2.imread(path)
    if image is None:
        return None

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    marker = detect_marker(image)
    if marker is None:
        return None

    marker_px, (x, y, r) = marker
    plant = detect_plant(hsv)
    if plant is None:
        return marker_px, 0, 0, image

    plant_contour, plant_px = plant
    scale = MARKER_REAL_AREA_CM2 / marker_px
    plant_cm2 = plant_px * scale

    # === Draw result ===
    out = image.copy()
    cv2.circle(out, (x, y), r, (255,0,0), 3)
    cv2.drawContours(out, [plant_contour], -1, (0,255,0), 4)
    cv2.putText(out, f"{plant_cm2:.2f} cm^2", (20,60),
                cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 4)

    return marker_px, plant_px, plant_cm2, out

In [ ]:
# Batch processing

results = []

for fname in os.listdir(IMAGE_DIR):
    if not fname.lower().endswith(('.jpg','.jpeg','.png')):
        continue

    fpath = os.path.join(IMAGE_DIR, fname)
    res = analyze_image(fpath)
    if res is None:
        print(f"{fname}: ❌ marker not detected")
        continue

    marker_px, plant_px, plant_cm2, img = res
    cv2.imwrite(os.path.join(OUTPUT_DIR, fname), img)

    results.append({
        'filename': fname,
        'marker_px': marker_px,
        'plant_px': plant_px,
        'plant_cm2': plant_cm2
    })

    print(f"{fname}: ✅ {plant_cm2:.2f} cm²")

In [ ]:
# Save CSV
pd.DataFrame(results).to_csv(os.path.join(OUTPUT_DIR, 'plant_area_results.csv'), index=False)
print("\n✅ Analysis complete")